In [2]:
import spacy
import asent
import pandas as pd

In [3]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("sentencizer")
nlp.add_pipe("asent_en_v1")


In [4]:
 def ProcessText(content):
        wordDict = {"NOUN": 0, "PROPN": 0, "VERB": 0, "ADJ": 0, "ADV": 0}
        uniqueTokens = set()
        sentimentalScore = None
        tokens = 0

        doc = nlp(content)
        tokens = [
            token.text.lower() for token in doc if token.pos_ in  wordDict.keys()
        ]

        tokens = [token for token in tokens if len(token) > 2]

     
        for token in doc:
            if token.pos_ in wordDict.keys():
                wordDict[token.pos_] += 1

        tokens_n = len(tokens)

        uniqueTokens.update(tokens)

        sentimentalScore = doc._.polarity

        return wordDict, uniqueTokens, sentimentalScore, tokens_n

In [5]:

def PrepareSingleDataText(title,wordDict, uniqueTokens, sentimentalScore, tokens, label):
        global counter
        total_w = wordDict["NOUN"] + wordDict["PROPN"] + wordDict["VERB"] + wordDict["ADV"] + wordDict["ADJ"]
        dataframe = {
            "title" : title,
            "neg_score": round(sentimentalScore.negative,3),
            "neu_score": round(sentimentalScore.neutral,3),
            "pos_score": round(sentimentalScore.positive,3),
            "compound_score": round(sentimentalScore.compound,3),
            "n_sentences": sentimentalScore.n_sentences,
            "n_tokens": tokens,
            "unique_tokens_r": round(len(uniqueTokens) / tokens,3),
            "nouns_r": round(wordDict["NOUN"] / total_w,3),
            "proper_nouns_r": round(wordDict["PROPN"] / total_w,3),
            "verbs_r": round(wordDict["VERB"] / total_w,3),
            "adverbs_r": round(wordDict["ADV"] / total_w,3),
            "adjectives_r": round(wordDict["ADJ"] / total_w,3),
            "news" : label
        }
        return dataframe



In [6]:
import multiprocessing as mp
import pandas as pd

def ProcessChunk(texts, label):
    results = []
    for text in texts:
        try:
            wordDict, uniqueTokens, sentimentalScore, tokens_n = ProcessText(text)
            data = PrepareSingleDataText(text[:50] + "...", wordDict,uniqueTokens, sentimentalScore, tokens_n, label)
            results.append(data)
        except Exception as e:
            print(f"Error processing text: {text[:50]}... Error: {str(e)}")
    return results

def CreateDatasetFromFile(file_path_1, file_path_2):
    with open(file_path_1, "r") as f:
        texts_1 = [t.strip() for t in f.read().split("\n") if t.strip()]
    with open(file_path_2, "r") as f:
        texts_2 = [t.strip() for t in f.read().split("\n") if t.strip()]

    with mp.Pool(processes=mp.cpu_count()) as pool:
        chunk_size = 1000  # Dostosuj tę wartość
        real_chunks = [texts_1[i:i+chunk_size] for i in range(0, len(texts_1), chunk_size)]
        fake_chunks = [texts_2[i:i+chunk_size] for i in range(0, len(texts_2), chunk_size)]

        real_results = pool.starmap(process_chunk, [(chunk, "Real") for chunk in real_chunks])
        fake_results = pool.starmap(process_chunk, [(chunk, "Fake") for chunk in fake_chunks])

    all_data = [item for sublist in real_results + fake_results for item in sublist]

    df = pd.DataFrame(all_data)
    df.set_index("title", inplace=True)
    df.to_csv("data.csv")
    

In [78]:
CreateDatasetFromFile("../data/real.txt", "../data/fake.txt")

Error processing text: and... Error: division by zero
Error processing text: and... Error: division by zero
Error processing text: 2016... Error: division by zero
Error processing text: 48413... Error: division by zero
Error processing text: 0 12... Error: division by zero
Error processing text: 65975... Error: division by zero
Error processing text: by 2016... Error: division by zero
Error processing text: In... Error: division by zero
Error processing text: for... Error: division by zero
Error processing text: 221... Error: division by zero
Error processing text: f... Error: division by zero
Error processing text: and m... Error: division by zero
Error processing text: 2016 by by... Error: division by zero
Error processing text: 66.1... Error: division by zero
Error processing text: 6416... Error: division by zero
Error processing text: 2016 by... Error: division by zero
Error processing text: up... Error: division by zero
Error processing text: no... Error: division by zero
Error pr

In [7]:
X_test

NameError: name 'X_test' is not defined